<a href="https://colab.research.google.com/github/FFI-Vietnam/camtrap-tools/blob/main/Wildlife%20Insights/bulk-upload/02_clean-deployment-dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This script is used for cleaning the deployments spreadsheet 
After runnning this script, these new files are added to 'data cleaning' folder

data cleaning
    |__ 2.1_clean_deployment_drop-column.csv
    |__ 2.2_clean_deployment_dropna-column.csv
    |__ 2.3_clean_deployment_format-datetime.csv
    
"""

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# mount with Google Drive to read and save file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# specifies Colab directories and file names
root = '/content/drive/'

dataset_folder = 'My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/dataset'
contain_folder = 'My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning'

deployment_dataset_file_name = 'Kon Plong 2019-20 camera trap deployment_v3.csv'
deployment_drop_column_file_name = "2.1_clean_deployment_drop-column.csv"
deployment_dropna_file_name = "2.2_clean_deployment_dropna.csv"
deployment_format_datetime_filename = "2.3_clean_deployment_format-datetime.csv"

In [4]:
# useful functions
def read_csv_Google_drive(root, contain_folder, file_name):
  file_path = os.path.join(root, contain_folder, file_name)
  return pd.read_csv(file_path)

def save_csv_Google_drive(df, root, contain_folder, file_name):
  """
  function to save a csv file to Google Drive
  param examples:
    root = '/content/drive/'
    contain_folder = 'My Drive/FFI/dataset'
    file_name = 'image_metadata(2020-06-26)_full.csv'
  """
  # save file to Colab runtime storage (will be deleted when this notebook is closed)
  df.to_csv('dataframe.csv', index=False)

  # save file back to Google Drive for permanent storage
  folder_path = os.path.join(root, contain_folder)
  file_path = os.path.join(root, contain_folder, file_name)
  try:
    os.makedirs(folder_path)
  except:
    pass

  with open('dataframe.csv', 'r') as f:
    df_file = f.read()

  with open(file_path, 'w') as f:
    f.write(df_file)

  print(f'File is saved to {file_name} in Google Drive at {file_path}')

In [ ]:
# read full dataset
df = read_csv_Google_drive(root, dataset_folder, deployment_dataset_file_name)
df.head()

,Station,Camera,Deployment Phase,NGO responsible,Station X (UTM 49N) - planned,Station Y (UTM 49N) - planned,Station X (UTM 49N) - actual,Station Y (UTM 49N) - actual,X station diff,Y station diff,Camera X (UTM 49N),Camera Y (UTM 49N),X camera diff,Y camera diff,Longitude,Latitude,Elevation(DEM),GPS accuracy (± m),Camera height,Camera orientation,Distance to pathway/road,Pathway/road width (cm),On/off road,Off road description,"Both cameras focused on same feature (no: 0, yes: 1)",Photo 5m,Photo 10m,Distance between 2 camera,Date setup,Time setup,Team setup,Date retrieved,Time retrieved,Team retrieval,Last image date-time (from R script),Camera still functioning at retrieval (Y/N)?,Date camera off,Time camera off,Number of trap nights,Notes
0,59,68824.0,1,NaN,195886,1659621,196117.0,1659635.0,231.0,14.0,196112.0,1659630.0,5.0,5.0,108.174406,14.994252,1309.0,NaN,26,202,300.0,30.0,0.0,NaN,0.0,1.0,1.0,8.0,24/04/2019,14:30,"An, Huy, A Lanh",8/24/2019,10:30,NaN,8/4/2019 13:10,N,04/08/2019,13:10,102,NaN
1,59,68704.0,1,NaN,195886,1659621,196117.0,1659635.0,231.0,14.0,196118.0,1659620.0,1.0,15.0,108.174463,14.994162,1313.0,NaN,23,357,NaN,NaN,0.0,NaN,0.0,1.0,1.0,8.0,24/04/2019,14:43,"An, Huy, A Lanh",8/24/2019,10:30,NaN,8/5/2019 18:44,N,05/08/2019,18:44,103,NaN
2,60,68885.0,1,NaN,198385,1659553,198549.0,1659945.0,164.0,392.0,198548.0,1659957.0,1.0,12.0,108.196998,14.997484,1096.0,14.0,20,99,NaN,NaN,0.0,1,0.0,1.0,0.0,10.0,24/04/2019,12:19,"Mạnh, A Liêm, A Hải",8/23/2019,16:10,NaN,7/31/2019 22:11,N,31/07/2019,22:11,98,NaN
3,60,67170.0,1,NaN,198385,1659553,198549.0,1659945.0,164.0,392.0,198559.0,1659949.0,10.0,4.0,108.197101,14.997413,1085.0,2.0,30,216,300.0,30.0,0.0,1,0.0,1.0,1.0,10.0,24/04/2019,12:16,"Mạnh, A Liêm, A Hải",8/23/2019,16:10,NaN,6/13/2019 7:00,N,13/06/2019,7:00,50,Images time-shifted forward to correct date-ti...
4,61,68966.0,1,NaN,200883,1659485,201068.0,1659553.0,185.0,68.0,201065.0,1659544.0,3.0,9.0,108.220431,14.994041,1174.0,2.0,34,76,NaN,NaN,0.0,1,0.0,1.0,1.0,14.5,25/04/2019,12:19,"Huy, An, Mạnh, A Liêm, A Lanh",8/27/2019,11:30,NaN,8/8/2019 7:43,N,08/08/2019,7:43,105,NaN


In [14]:
# 2.1) keep important columns

df = read_csv_Google_drive(root, dataset_folder, deployment_dataset_file_name)
 
df = df[['Station', 
         'Camera', 
         'Deployment Phase', 
         'Longitude', 
         'Latitude', 
         'Date setup', 
         'Time setup', 
         'Last image date-time (from R script)', # note the format is MM/DD/YYYY hh:mm
         'Date retrieved', # to replace null date-time camera off
         'Time retrieved',
         'Camera still functioning at retrieval (Y/N)?'
        ]]

# save to Google Drive
save_csv_Google_drive(df, root, contain_folder, deployment_drop_column_file_name)

df.head()

File is saved to 2.1_clean_deployment_drop-column.csv in Google Drive at /content/drive/My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning/2.1_clean_deployment_drop-column.csv


,Station,Camera,Deployment Phase,Longitude,Latitude,Date setup,Time setup,Last image date-time (from R script),Date retrieved,Time retrieved,Camera still functioning at retrieval (Y/N)?
0,59,68824.0,1,108.174406,14.994252,24/04/2019,14:30,8/4/2019 13:10,8/24/2019,10:30,N
1,59,68704.0,1,108.174463,14.994162,24/04/2019,14:43,8/5/2019 18:44,8/24/2019,10:30,N
2,60,68885.0,1,108.196998,14.997484,24/04/2019,12:19,7/31/2019 22:11,8/23/2019,16:10,N
3,60,67170.0,1,108.197101,14.997413,24/04/2019,12:16,6/13/2019 7:00,8/23/2019,16:10,N
4,61,68966.0,1,108.220431,14.994041,25/04/2019,12:19,8/8/2019 7:43,8/27/2019,11:30,N


In [16]:
# 2.2) remove null rows
df = read_csv_Google_drive(root, contain_folder, deployment_drop_column_file_name)

df = df.dropna(how='all', subset=['Camera'])

# save to Google Drive
save_csv_Google_drive(df, root, contain_folder, deployment_dropna_file_name)

df.isnull().sum()

File is saved to 2.2_clean_deployment_dropna.csv in Google Drive at /content/drive/My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning/2.2_clean_deployment_dropna.csv


Station                                           0
Camera                                            0
Deployment Phase                                  0
Longitude                                         0
Latitude                                          0
Date setup                                        0
Time setup                                        3
Last image date-time (from R script)              7
Date retrieved                                    0
Time retrieved                                  206
Camera still functioning at retrieval (Y/N)?      0
dtype: int64

In [21]:
# 2.3) format datetime
# datetime must be formatted into yyyy-mm-dd hh:mm:ss

def format_datetime_setup(date, time):
  """
  ex: 
    date = 24/04/2019
    time = 14:30
    --> return 2019-04-24 14:30:00 
  """
  DD, MM, YYYY = date.split('/')
  # in case no time, set default 12:00
  try:
    hh, mm = time.split(':')
  except:
    hh, mm = '12', '00'
  # pad leading 0
  MM, DD, hh, mm, ss = MM.zfill(2), DD.zfill(2), hh.zfill(2), mm.zfill(2), '00'
  return f'{YYYY}-{MM}-{DD} {hh}:{mm}:{ss}'
 

def format_datetime_retrieval(dt):
  """
  format datetime from MM/DD/YYYY hh:mm into YYYY-MM-DD hh:mm:ss
  """
  try:
    hh, mm = dt.split()[1].split(':')
  except:
    hh, mm = "12", "00"
  try:
    MM, DD, YYYY = dt.split()[0].split('/')
    # pad leading 0
    MM, DD, hh, mm, ss = MM.zfill(2), DD.zfill(2), hh.zfill(2), mm.zfill(2), '00'
    return f'{YYYY}-{MM}-{DD} {hh}:{mm}:{ss}'
  except:
    return 'error'


df = read_csv_Google_drive(root, contain_folder, deployment_dropna_file_name)

datetime_setup = []
datetime_retrieval = []
datetime_for_null_camera_off = []

for i in df.index:

  datetime_setup.append(format_datetime_setup(df.loc[i, 'Date setup'], 
                                              df.loc[i, 'Time setup']))
  datetime_retrieval.append(format_datetime_retrieval(df.loc[i, 'Last image date-time (from R script)']))
  datetime_for_null_camera_off.append(format_datetime_retrieval(f"{df.loc[i, 'Date retrieved']} {df.loc[i, 'Time retrieved']}"))

df['Datetime setup'] = datetime_setup
df['Datetime retrieval'] = datetime_retrieval
df['Datetime for null camera-off'] = datetime_for_null_camera_off

# save to Google Drive
save_csv_Google_drive(df, root, contain_folder, deployment_format_datetime_filename)
df

File is saved to 2.3_clean_deployment_format-datetime.csv in Google Drive at /content/drive/My Drive/FFI/Wildlife Insights Bulk Upload Test/bulk-upload_template-autofill/data cleaning/2.3_clean_deployment_format-datetime.csv


,Station,Camera,Deployment Phase,Longitude,Latitude,Date setup,Time setup,Last image date-time (from R script),Date retrieved,Time retrieved,Camera still functioning at retrieval (Y/N)?,Datetime setup,Datetime retrieval,Datetime for null camera-off
0,59,68824.0,1,108.174406,14.994252,24/04/2019,14:30,8/4/2019 13:10,8/24/2019,10:30,N,2019-04-24 14:30:00,2019-08-04 13:10:00,2019-08-24 10:30:00
1,59,68704.0,1,108.174463,14.994162,24/04/2019,14:43,8/5/2019 18:44,8/24/2019,10:30,N,2019-04-24 14:43:00,2019-08-05 18:44:00,2019-08-24 10:30:00
2,60,68885.0,1,108.196998,14.997484,24/04/2019,12:19,7/31/2019 22:11,8/23/2019,16:10,N,2019-04-24 12:19:00,2019-07-31 22:11:00,2019-08-23 16:10:00
3,60,67170.0,1,108.197101,14.997413,24/04/2019,12:16,6/13/2019 7:00,8/23/2019,16:10,N,2019-04-24 12:16:00,2019-06-13 07:00:00,2019-08-23 16:10:00
4,61,68966.0,1,108.220431,14.994041,25/04/2019,12:19,8/8/2019 7:43,8/27/2019,11:30,N,2019-04-25 12:19:00,2019-08-08 07:43:00,2019-08-27 11:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,433,69002.0,2,108.460331,14.558804,21/10/2019,12:11,12/18/2019 12:47,12/20/2019,NaN,N,2019-10-21 12:11:00,2019-12-18 12:47:00,2019-12-20 12:00:00
238,434,68962.0,2,108.490764,14.561111,21/10/2019,10:03,11/3/2019 17:25,12/24/2019,NaN,N,2019-10-21 10:03:00,2019-11-03 17:25:00,2019-12-24 12:00:00
239,434,68988.0,2,108.490727,14.561110,21/10/2019,10:15,12/24/2019 13:07,12/24/2019,NaN,Y,2019-10-21 10:15:00,2019-12-24 13:07:00,2019-12-24 12:00:00
240,435,68998.0,2,108.513166,14.560067,21/10/2019,13:09,12/23/2019 12:44,12/23/2019,NaN,Y,2019-10-21 13:09:00,2019-12-23 12:44:00,2019-12-23 12:00:00
